<a href="https://colab.research.google.com/github/Blackan06/DATA_MINING_C2_LAB/blob/main/DATA_MINING_C2_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import pandas as pd
import numpy as np
from google.colab import files

#Load Data In Labor Excel

In [123]:
labor_data = pd.read_excel('/content/Labor.xlsx', header=None)

In [124]:
print(labor_data.head())


  0    1    2  3    4   5            6  7  8     9   10              11    12  \
0  1    5    ?  ?    ?  40            ?  ?  2     ?  11        average'     ?   
1  2  4.5  5.8  ?    ?  35    ret_allw'  ?  ?  yes'  11  below_average'     ?   
2  ?    ?    ?  ?    ?  38  empl_contr'  ?  5     ?  11       generous'  yes'   
3  3  3.7    4  5  tc'   ?            ?  ?  ?  yes'   ?               ?     ?   
4  3  4.5  4.5  5    ?  40            ?  ?  ?     ?  12        average'     ?   

      13    14     15     16  
0      ?  yes'      ?  good'  
1  full'     ?  full'  good'  
2  half'  yes'  half'  good'  
3      ?  yes'      ?  good'  
4  half'  yes'  half'  good'  


In [125]:
column_names = [
    "duration", "wage1", "wage2", "wage3", "cola", "hours", "pension",
    "standby_pay", "shift_diff", "educ_allow", "holidays", "vacation",
    "longterm_disability", "dental_insurance", "bereavement", "health_plan", "class"
]
labor_data.columns = column_names

In [126]:
labor_data.head(5)

,duration,wage1,wage2,wage3,cola,hours,pension,standby_pay,shift_diff,educ_allow,holidays,vacation,longterm_disability,dental_insurance,bereavement,health_plan,class
0,1,5,?,?,?,40,?,?,2,?,11,average',?,?,yes',?,good'
1,2,4.5,5.8,?,?,35,ret_allw',?,?,yes',11,below_average',?,full',?,full',good'
2,?,?,?,?,?,38,empl_contr',?,5,?,11,generous',yes',half',yes',half',good'
3,3,3.7,4,5,tc',?,?,?,?,yes',?,?,?,?,yes',?,good'
4,3,4.5,4.5,5,?,40,?,?,?,?,12,average',?,half',yes',half',good'


In [127]:
labor_data_cleaned = labor_data.replace('?', np.nan)


<ipython-input-127-217e3cb4fc9c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labor_data_cleaned = labor_data.replace('?', np.nan)


#Using method: Fill in missing values ​​with mean

Since these attributes are numeric data, it makes sense to use the average of the remaining values ​​to fill in the blank cells. This ensures that added values ​​do not change the nature of the data too much and helps keep the overall trend of each attribute intact.

In [128]:
column_names = [
    "duration", "wage1", "wage2", "wage3", "cola", "hours", "pension",
    "stby_pay", "shift_diff", "educ_allow", "holidays", "vacation",
    "longterm_disability", "dental_insurance", "bereavement", "health_plan", "class"
]
labor_data_cleaned.columns = column_names

In [129]:
integer_constraints = {
    'duration': (1, 7),
    'hours': (35, 40),
    'stby_pay': (2, 25),
    'shift_diff': (1, 25),
    'holidays': (9, 15),
}

float_constraints = {
    'wage1': (2.0, 7.0),
    'wage2': (2.0, 7.0),
    'wage3': (2.0, 7.0),
}

In [130]:
# Handle missing values for integer columns
for col, (min_val, max_val) in integer_constraints.items():
    labor_data_cleaned[col] = pd.to_numeric(labor_data_cleaned[col], errors='coerce')
    # Replace missing values with the median or a reasonable value in the range
    labor_data_cleaned[col] = labor_data_cleaned[col].fillna(
        int(labor_data_cleaned[col].clip(lower=min_val, upper=max_val).mean())
    ).astype(int)
    # Ensure values fall within valid range
    labor_data_cleaned[col] = labor_data_cleaned[col].clip(lower=min_val, upper=max_val)

# Handle missing values for float columns
for col, (min_val, max_val) in float_constraints.items():
    labor_data_cleaned[col] = pd.to_numeric(labor_data_cleaned[col], errors='coerce')
    # Replace missing values with the mean within valid range
    labor_data_cleaned[col] = labor_data_cleaned[col].fillna(
        labor_data_cleaned[col].clip(lower=min_val, upper=max_val).mean()
    ).astype(float)
    # Ensure values fall within valid range
    labor_data_cleaned[col] = labor_data_cleaned[col].clip(lower=min_val, upper=max_val)

#Using method: Fill in missing values ​​with mode

Since these attributes are categories, filling in missing values ​​with mode is a suitable choice. Mode is the value that appears most often and is considered representative of that attribute. This method helps maintain the integrity of the data without distorting its characteristics.

In [131]:
categorical_cols = {
    'cola': ['none', 'tcf', 'tc'],
    'pension': ['none', 'ret_allw', 'empl_contr'],
    'vacation': ['ba', 'avg', 'gnr'],
    'dental_insurance': ['none', 'half', 'full'],
    'health_plan': ['none', 'half', 'full'],
    'educ_allow': ['yes','no'],
    'longterm_disability': ['yes','no'],
    'bereavement': ['yes','no']
}

# Remove any apostrophes in the categorical columns and fill missing values with mode
for col in categorical_cols:
    if col in labor_data_cleaned.columns:
        labor_data_cleaned[col] = labor_data_cleaned[col].apply(lambda x: str(x).replace("'", '') if pd.notna(x) else x)

        labor_data_cleaned[col] = labor_data_cleaned[col].astype(str)
        labor_data_cleaned[col].replace('nan', pd.NA, inplace=True)

        if not labor_data_cleaned[col].mode().empty:
            mode_value = labor_data_cleaned[col].mode()[0]
            if pd.notna(mode_value):
                labor_data_cleaned[col].fillna(mode_value, inplace=True)

<ipython-input-131-6e3ff8be0183>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labor_data_cleaned[col].replace('nan', pd.NA, inplace=True)
<ipython-input-131-6e3ff8be0183>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

#Clean data in column class

Remove   "'" from data

In [132]:
labor_data_cleaned['class'] = labor_data_cleaned['class'].str.replace("'", "")

labor_data_cleaned['class'].fillna(labor_data_cleaned['class'].mode()[0], inplace=True)

<ipython-input-132-edc0282702a0>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labor_data_cleaned['class'].fillna(labor_data_cleaned['class'].mode()[0], inplace=True)


In [133]:
missing_summary = labor_data_cleaned.isnull().sum()
print("Số lượng giá trị thiếu sau khi xử lý:")
print(missing_summary)

Số lượng giá trị thiếu sau khi xử lý:
duration               0
wage1                  0
wage2                  0
wage3                  0
cola                   0
hours                  0
pension                0
stby_pay               0
shift_diff             0
educ_allow             0
holidays               0
vacation               0
longterm_disability    0
dental_insurance       0
bereavement            0
health_plan            0
class                  0
dtype: int64


In [134]:
labor_data_cleaned.round(1).head()

,duration,wage1,wage2,wage3,cola,hours,pension,stby_pay,shift_diff,educ_allow,holidays,vacation,longterm_disability,dental_insurance,bereavement,health_plan,class
0,1,5.0,4.0,3.9,none,40,empl_contr,7,2,no,11,average,yes,half,yes,full,good
1,2,4.5,5.8,3.9,none,35,ret_allw,7,4,yes,11,below_average,yes,full,yes,full,good
2,2,3.8,4.0,3.9,none,38,empl_contr,7,5,no,11,generous,yes,half,yes,half,good
3,3,3.7,4.0,5.0,tc,38,empl_contr,7,4,yes,11,below_average,yes,half,yes,full,good
4,3,4.5,4.5,5.0,none,40,empl_contr,7,4,no,12,average,yes,half,yes,half,good


In [135]:
labor_data_cleaned.round(1).to_excel('labor_data_cleaned.xlsx', index=False)

files.download('labor_data_cleaned.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
labor_data.to_excel('labor_data.xlsx', index=False)

files.download('labor_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>